In [34]:
!pip  install --upgrade --quiet neo4j openai pandas rdflib-neo4j langchain_openai langchain_nvidia_ai_endpoints  langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.6 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["NEO4J_URI"] = "bolt://44.203.231.196:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "brass-barge-cells"
os.environ["NVIDIA_API_KEY"] = userdata.get('NVIDIA_API_KEY')

In [13]:
croassant_file_path = 'https://raw.githubusercontent.com/fabischkamau/https---github.com-fabischkamau-Knowledge-Graphs/main/dataset/supply-chain-analysis-metadata.json'
data_file_path = 'https://raw.githubusercontent.com/fabischkamau/https---github.com-fabischkamau-Knowledge-Graphs/main/dataset/supply_chain_data.csv'


In [11]:

from rdflib_neo4j import Neo4jStoreConfig, Neo4jStore, HANDLE_VOCAB_URI_STRATEGY
from rdflib import Graph

auth_data = {'uri': os.getenv("NEO4J_URI"),
             'database': "neo4j",
             'user': os.getenv("NEO4J_USERNAME"),
             'pwd': os.getenv("NEO4J_PASSWORD")
             }

config = Neo4jStoreConfig(auth_data=auth_data,
                          handle_vocab_uri_strategy=HANDLE_VOCAB_URI_STRATEGY.IGNORE,
                          batching=True)

graph_store = Graph(store=Neo4jStore(config=config))

#temporary hack to parse JSON-LD until new version of rdflib-neo4j released
temp = Graph()
temp.parse(croassant_file_path, format="json-ld")

graph_store.parse(data=temp.serialize(format="ttl"), format="ttl")
graph_store.close(True)


Uniqueness constraint on :Resource(uri) is created.
The store is now: Open
The store is now: Closed
IMPORTED 267 TRIPLES


In [12]:
"""match graph = (:Dataset)-[:recordSet]->(:RecordSet)-[:field]->(:Field)-[:dataType]->()
return graph"""

'match graph = (:Dataset)-[:recordSet]->(:RecordSet)-[:field]->(:Field)-[:dataType]->()\nreturn graph'

In [4]:

from neo4j import GraphDatabase

with GraphDatabase.driver(os.getenv("NEO4J_URI"), auth=("neo4j", os.getenv("NEO4J_PASSWORD"))) as driver:
    driver.verify_connectivity()

    features_query = """
    match (d:Dataset)-[:recordSet]->(rs:RecordSet)-[:field]->(f:Field)-[:dataType]->(dt),
          (f)-[:source]->()-[:extract]->(e)
    return d.name as datasetname, d.description as datasetdescription,
           collect ({featurename : e.column, datatype : dt.uri }) as datasetfeatures
    """
    records, summary, keys = driver.execute_query(features_query)

    result = records[0]

    ds_name = result['datasetname']
    ds_description = result['datasetdescription']
    ds_features = result['datasetfeatures']


In [5]:

system = "You are a data modelling expert capable of creating high quality entity-relationship models from flat datasets"

prompt=f"""

From the list of features in the following dataset create a list of entities and relationships with their
attributes in a simple json format and map them to the features in the dataset.

The attributes don't need to be named after the features in the dataset, but they should be mapped to the corresponding feature name.
No extra text or comments, only the json as output without the string quotation.
Add mappedTo to attributes

DATASET NAME: {ds_name}

DATASET DESCRIPTION: {ds_description}

DATASET FEATURES: {ds_features}
"""

print(prompt)




From the list of features in the following dataset create a list of entities and relationships with their
attributes in a simple json format and map them to the features in the dataset.

The attributes don't need to be named after the features in the dataset, but they should be mapped to the corresponding feature name.
No extra text or comments, only the json as output without the string quotation.
Add mappedTo to attributes

DATASET NAME: Supply Chain Analysis

DATASET DESCRIPTION: Supply chain analytics is a valuable part of data-driven decision-making in various industries such as manufacturing, retail, healthcare, and logistics. It is the process of collecting, analyzing and interpreting data related to the movement of products and services from suppliers to customers.

Here is a dataset we collected from a Fashion and Beauty startup. The dataset is based on the supply chain of Makeup products. Below are all the features in the dataset:

- Product Type
- SKU
- Price
- Availabilit

In [6]:

from openai import OpenAI

client = OpenAI()
completion = client.chat.completions.create(model="gpt-4o-mini",
                                                  temperature=0,
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": prompt},
        ])


In [7]:

genschema = completion.choices[0].message.content
print(genschema)

{
  "entities": [
    {
      "name": "Product",
      "attributes": [
        {
          "name": "productType",
          "mappedTo": "Product Type"
        },
        {
          "name": "sku",
          "mappedTo": "SKU"
        },
        {
          "name": "price",
          "mappedTo": "Price"
        },
        {
          "name": "availability",
          "mappedTo": "Availability"
        },
        {
          "name": "numberOfProductsSold",
          "mappedTo": "Number of products sold"
        },
        {
          "name": "revenueGenerated",
          "mappedTo": "Revenue generated"
        },
        {
          "name": "stockLevels",
          "mappedTo": "Stock levels"
        },
        {
          "name": "orderQuantities",
          "mappedTo": "Order quantities"
        }
      ]
    },
    {
      "name": "Customer",
      "attributes": [
        {
          "name": "demographics",
          "mappedTo": "Customer demographics"
        }
      ]
    },
    {
   

In [8]:
e_name = "name"
a_name = "name"
map_att = "mappedTo"
r_name = "type"
r_from = "from"
r_to = "to"


def getIdValExprFor(sch, ename):
  result = []
  for x in sch["entities"]:
    if x[e_name] == ename:
      for y in x["attributes"]:
        result.append("toString(record.`" + y[map_att] + "`)")
  return " apoc.hashing.fingerprint( " + " + ".join(result) + " )"



def generateImportScript(sch):

  constraints = {}
  cypher_import = {}

  for e in sch["entities"]:
      constraints[e[e_name]] = "CREATE CONSTRAINT IF NOT EXISTS FOR (n:" + e[e_name] + ") REQUIRE n._id IS UNIQUE; "
      cypher = []
      cypher.append("unwind $records AS record")
      cypher.append("merge (n:`" + e[e_name] + "` { _id: " +  getIdValExprFor(sch, e[e_name]) + "} )")
      for p in e["attributes"]:
        propName = p[a_name]
        #type conversion ?
        #propType = p["datatype"]
        cypher.append("set n.`" + propName + "` = record.`" + p[map_att] + "`")
      cypher.append("return count(*) as total ;")
      cypher_import[e[e_name]] = ' \n'.join(cypher)

  for r in sch["relationships"]:
    cypher = []
    cypher.append("unwind $records AS record")
    cypher.append("match (source:" + r[r_from] + " { _id : " + getIdValExprFor(sch, r[r_from]) + "} )")
    cypher.append("match (target:" + r[r_to] + " { _id : " + getIdValExprFor(sch, r[r_to]) + "} )")
    cypher.append("merge (source)-[r:`"+ r[r_name] +"`]->(target)")
    cypher.append("return count(*) as total")
    cypher_import[r[r_name]] = ' \n'.join(cypher)

  return cypher_import , constraints

In [9]:
cypher_import, constraints = generateImportScript(eval(genschema))

In [10]:
constraints

{'Product': 'CREATE CONSTRAINT IF NOT EXISTS FOR (n:Product) REQUIRE n._id IS UNIQUE; ',
 'Customer': 'CREATE CONSTRAINT IF NOT EXISTS FOR (n:Customer) REQUIRE n._id IS UNIQUE; ',
 'Supplier': 'CREATE CONSTRAINT IF NOT EXISTS FOR (n:Supplier) REQUIRE n._id IS UNIQUE; ',
 'Shipping': 'CREATE CONSTRAINT IF NOT EXISTS FOR (n:Shipping) REQUIRE n._id IS UNIQUE; ',
 'QualityControl': 'CREATE CONSTRAINT IF NOT EXISTS FOR (n:QualityControl) REQUIRE n._id IS UNIQUE; ',
 'CostAnalysis': 'CREATE CONSTRAINT IF NOT EXISTS FOR (n:CostAnalysis) REQUIRE n._id IS UNIQUE; ',
 'LeadTime': 'CREATE CONSTRAINT IF NOT EXISTS FOR (n:LeadTime) REQUIRE n._id IS UNIQUE; '}

In [11]:
cypher_import

{'Product': 'unwind $records AS record \nmerge (n:`Product` { _id:  apoc.hashing.fingerprint( toString(record.`Product Type`) + toString(record.`SKU`) + toString(record.`Price`) + toString(record.`Availability`) + toString(record.`Number of products sold`) + toString(record.`Revenue generated`) + toString(record.`Stock levels`) + toString(record.`Order quantities`) )} ) \nset n.`productType` = record.`Product Type` \nset n.`sku` = record.`SKU` \nset n.`price` = record.`Price` \nset n.`availability` = record.`Availability` \nset n.`numberOfProductsSold` = record.`Number of products sold` \nset n.`revenueGenerated` = record.`Revenue generated` \nset n.`stockLevels` = record.`Stock levels` \nset n.`orderQuantities` = record.`Order quantities` \nreturn count(*) as total ;',
 'Customer': 'unwind $records AS record \nmerge (n:`Customer` { _id:  apoc.hashing.fingerprint( toString(record.`Customer demographics`) )} ) \nset n.`demographics` = record.`Customer demographics` \nreturn count(*) as 

In [14]:

print("\n\n#CONSTRAINTS:\n")
for q in constraints.keys():
  print(q + ": \n"+ constraints[q] + "\n")

print("#\n\nLOADERS:\n")
for q in cypher_import.keys():
  print(q + ": \n\nfile: " + data_file_path + "\n\n"+ cypher_import[q] + "\n\n")



#CONSTRAINTS:

Product: 
CREATE CONSTRAINT IF NOT EXISTS FOR (n:Product) REQUIRE n._id IS UNIQUE; 

Customer: 
CREATE CONSTRAINT IF NOT EXISTS FOR (n:Customer) REQUIRE n._id IS UNIQUE; 

Supplier: 
CREATE CONSTRAINT IF NOT EXISTS FOR (n:Supplier) REQUIRE n._id IS UNIQUE; 

Shipping: 
CREATE CONSTRAINT IF NOT EXISTS FOR (n:Shipping) REQUIRE n._id IS UNIQUE; 

QualityControl: 
CREATE CONSTRAINT IF NOT EXISTS FOR (n:QualityControl) REQUIRE n._id IS UNIQUE; 

CostAnalysis: 
CREATE CONSTRAINT IF NOT EXISTS FOR (n:CostAnalysis) REQUIRE n._id IS UNIQUE; 

LeadTime: 
CREATE CONSTRAINT IF NOT EXISTS FOR (n:LeadTime) REQUIRE n._id IS UNIQUE; 

#

LOADERS:

Product: 

file: https://raw.githubusercontent.com/fabischkamau/https---github.com-fabischkamau-Knowledge-Graphs/main/dataset/supply_chain_data.csv

unwind $records AS record 
merge (n:`Product` { _id:  apoc.hashing.fingerprint( toString(record.`Product Type`) + toString(record.`SKU`) + toString(record.`Price`) + toString(record.`Availabilit

In [22]:
import pandas as pd
df = pd.read_csv(data_file_path)
df.head()

,Product type,SKU,Price,Availability,Number of products sold,Revenue generated,Customer demographics,Stock levels,Lead times,Order quantities,...,Location,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Inspection results,Defect rates,Transportation modes,Routes,Costs
0,haircare,SKU0,69.808006,55,802,8661.996792,Non-binary,58,7,96,...,Mumbai,29,215,29,46.279879,Pending,0.226410,Road,Route B,187.752075
1,skincare,SKU1,14.843523,95,736,7460.900065,Female,53,30,37,...,Mumbai,23,517,30,33.616769,Pending,4.854068,Road,Route B,503.065579
2,haircare,SKU2,11.319683,34,8,9577.749626,Unknown,1,10,88,...,Mumbai,12,971,27,30.688019,Pending,4.580593,Air,Route C,141.920282
3,skincare,SKU3,61.163343,68,83,7766.836426,Non-binary,23,13,59,...,Kolkata,24,937,18,35.624741,Fail,4.746649,Rail,Route A,254.776159
4,skincare,SKU4,4.805496,26,871,2686.505152,Non-binary,5,3,56,...,Delhi,5,414,3,92.065161,Fail,3.145580,Air,Route A,923.440632


In [29]:
def insert_data(session, query, frame):
  res = session.execute_write(query, parameters = {"records": frame.to_dict('records')})
#

In [32]:
import pandas as pd
from neo4j import GraphDatabase, basic_auth


with GraphDatabase.driver(os.getenv("NEO4J_URI"), auth=("neo4j", os.getenv("NEO4J_PASSWORD"))) as driver:
    driver.verify_connectivity()

    for c in constraints.keys():
      driver.execute_query(constraints[c])

    session = driver.session(database="neo4j")
    print("Source file: " + data_file_path)
    for q in cypher_import.keys():
      print("importing " + q )
      df = pd.read_csv(data_file_path, encoding = "ISO-8859-1")
      result = session.execute_write(lambda tx: tx.run(cypher_import[q], records = df.to_dict('records')))
      print(result)
    session.close()



Source file: https://raw.githubusercontent.com/fabischkamau/https---github.com-fabischkamau-Knowledge-Graphs/main/dataset/supply_chain_data.csv
importing Product
importing Customer
importing Supplier
importing Shipping
importing QualityControl
importing CostAnalysis
importing LeadTime
importing suppliedBy
importing purchasedBy
importing shippedVia
importing undergoes
importing analyzedBy
importing has


In [33]:
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]=userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_PROJECT"]="pr-shadowy-latex-56"

In [35]:
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI
from google.colab import userdata
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.prompts import ChatPromptTemplate
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["NEO4J_URI"] = "bolt://44.203.231.196:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "brass-barge-cells"
os.environ["NVIDIA_API_KEY"] = userdata.get('NVIDIA_API_KEY')


llmOllama = ChatNVIDIA(model="meta/llama-3.1-70b-instruct")
graph = Neo4jGraph()
llm = ChatOpenAI(temperature=0 ,model="gpt-3.5-turbo")

In [37]:



graph_prompt = ChatPromptTemplate.from_messages([
  ("system",
   f"""
    You are a Cypher Expert Generating Cypher Queries to query a Neo4j Database from User Questions.\n

    Task:
      *  Use the schema provided to generate Cypher Queries.\n
      *  Strictly follow the schema while generating Cypher Queries.\n
      *  Do not return any explanations or apologies in your responses.\n
      *  Do not return anything other than cypher queries.
      * use fullnames for cypher variables

    """),
    ("human","Schema: {schema}"),
    ("human","The question is: {query}") ,

  ]
)

qa_prompt = ChatPromptTemplate.from_messages([
    ("system","""
    You are a Chat Assistant helping users with their queries.
    Task: You are to generate answers to the Question asked using the result from context provided. Be concise and provide all relevant information from context.

    Instructions:
    - Read the question and the context again before answering.
    - Use only the provided context to answer the question.
    - If no context is provided, explain to the user you could not find that information.
    - Do not fall back to your pre-trained knowledge.
    - Do not mention context in your answer.
    - Be as helpful as you can.
    - Do not hallucinate.
    - Make sure to provide answer from context before saying you don't know.



    Note: The context provided comes from an Authoritative and is a fact source do not doubt it .
    Note: Do not fallback to your pre-trained knowledge.
    Note: Provide the answer an do not mention you are providing the answer from the context.

    *Be friendly with your with your tone.
    """),

    ("human","Question:{question}"),
    ("human","Context:{context}"),
])

graphchain = GraphCypherQAChain.from_llm(
    llm=llmOllama, qa_llm=llm,  graph=graph, verbose=True, cypher_prompt=graph_prompt, return_intermediate_steps=True, qa_prompt=qa_prompt
)

In [38]:
results = graphchain.invoke({"query":"Find the most popular supplier"})
print(results["result"])



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Product)-[:suppliedBy]->(s:Supplier) RETURN s.name as name, COUNT(DISTINCT p.sku) as count ORDER BY count DESC LIMIT 1
Full Context:
[{'name': 'Supplier 3', 'count': 1}]

> Finished chain.
Supplier 3 is the most popular supplier.
